In [1]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
from scipy.sparse.linalg import svds
import pickle
import surprise as srp

In [2]:
# path to dataset file
#file_path = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.data')

# As we're loading a custom dataset, we need to define a reader. In the
# movielens-100k dataset, each line has the following format:
# 'user item rating timestamp', separated by '\t' characters.
# and split it into 3 folds for cross-validation.
reader = srp.Reader(line_format='user item rating timestamp', sep='::')

data = srp.Dataset.load_from_file('ml-1m/ratings.dat', reader=reader)
data.split(n_folds=3)

# We'll use the famous SVD algorithm.
algo = srp.SVDpp()

# Evaluate performances of our algorithm on the dataset.
perf = srp.evaluate(algo, data, measures=['RMSE', 'MAE'])

Evaluating RMSE, MAE of algorithm SVDpp.

------------
Fold 1
RMSE: 0.8693
MAE:  0.6786
------------
Fold 2
RMSE: 0.8719
MAE:  0.6810
------------
Fold 3
RMSE: 0.8721
MAE:  0.6824
------------
------------
Mean RMSE: 0.8711
Mean MAE : 0.6806
------------
------------


In [3]:
print(perf)

defaultdict(<class 'list'>, {'rmse': [0.8693114846265434, 0.87192132094947505, 0.87211527057852101], 'mae': [0.67856991520033227, 0.68100048776613697, 0.68235689322194915]})
